In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/")
%cd /content/drive/MyDrive/new-york-city-taxi-fare-prediction
!ls



/content/drive/MyDrive/new-york-city-taxi-fare-prediction
'DS1019-proposal .gslides'      GCP-Coupons-Instructions.rtf.gdoc   train.csv
 final_code.ipynb	        sample_submission.csv
 GCP-Coupons-Instructions.rtf   test.csv


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"zyttiw","key":"d2c10c50b5fccb516de3331dcd225b09"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c new-york-city-taxi-fare-prediction

 99% 1.55G/1.56G [00:13<00:00, 116MB/s]
100% 1.56G/1.56G [00:13<00:00, 121MB/s]


In [ ]:
!unzip /content/new-york-city-taxi-fare-prediction.zip -d /content

Archive:  /content/new-york-city-taxi-fare-prediction.zip
  inflating: /content/GCP-Coupons-Instructions.rtf  
  inflating: /content/sample_submission.csv  
  inflating: /content/test.csv       
  inflating: /content/train.csv      


In [ ]:
train = pd.read_csv('/content/train.csv')

In [ ]:
import pandas as pd 
import dask.dataframe as dd
import os
from tqdm import tqdm

TRAIN_PATH = '/content/train.csv'

In [ ]:
%%time
# Assume we only know that the csv file is somehow large, but not the exact size
# we want to know the exact number of rows

# Method 1, using file.readlines. Takes about 20 seconds.
with open(TRAIN_PATH) as file:
    n_rows = len(file.readlines())

print (f'Exact number of rows: {n_rows}')

Exact number of rows: 55423857
CPU times: user 12.3 s, sys: 6.37 s, total: 18.7 s
Wall time: 25.1 s


In [ ]:
df_tmp = pd.read_csv(TRAIN_PATH, nrows=5)
df_tmp.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [ ]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                5 non-null      object 
 1   fare_amount        5 non-null      float64
 2   pickup_datetime    5 non-null      object 
 3   pickup_longitude   5 non-null      float64
 4   pickup_latitude    5 non-null      float64
 5   dropoff_longitude  5 non-null      float64
 6   dropoff_latitude   5 non-null      float64
 7   passenger_count    5 non-null      int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 448.0+ bytes


In [ ]:
traintypes = {'key':'object',
    'fare_amount': 'float64',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

cols = list(traintypes.keys())